In [2]:
from data import load_data
import matplotlib.pyplot as plt
import os
import numpy as np

import time

from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.model_selection import train_test_split

from keras.models import Model
from keras.layers.wrappers import TimeDistributed, Bidirectional
from keras.layers.recurrent import GRU
from keras.layers import Input
from keras.layers.convolutional import Conv2D, MaxPooling2D, ZeroPadding3D
from keras.layers.core import Lambda, Dropout, Flatten, Dense, Activation
from keras.optimizers import Adam
from keras import backend as K

CURRENT_PATH = '/home/ubuntu/assignments/machine-lip-reading/preprocessing'
DATA_PATH = CURRENT_PATH + '/../data'

def ctc_lambda_func(args):
    import tensorflow as tf
    y_pred, labels, input_length, label_length = args
    # From Keras example image_ocr.py:
    # the 2 is critical here since the first couple outputs of the RNN
    # tend to be garbage:
    # y_pred = y_pred[:, 2:, :]
    label_length = K.cast(tf.squeeze(label_length),'int32')
    input_length = K.cast(tf.squeeze(input_length),'int32')
    labels = K.ctc_label_dense_to_sparse(labels, label_length)
    #y_pred = y_pred[:, :, :]
    #return K.ctc_batch_cost(labels, y_pred, input_length, label_length, ignore_longer_outputs_than_inputs=True)
    return tf.nn.ctc_loss(labels, y_pred, input_length, ctc_merge_repeated=False,
                         ignore_longer_outputs_than_inputs = True, time_major = False)
def CTC(name, args):
	return Lambda(ctc_lambda_func, output_shape=(1,), name=name)(args)


def build_model(input_size, output_size = 28, max_string_len = 10):
    # model = Sequential()
    input_data = Input(name='the_input', shape=input_size, dtype='float32')
    x = ZeroPadding3D(padding=(0,2,2), name='padding1')(input_data)
    x = TimeDistributed(Conv2D(filters = 32, kernel_size = 5, strides = (2,2),
                             padding = 'same', activation = 'relu'))(x)
    print
    x = TimeDistributed(MaxPooling2D(pool_size=(2,2), strides=None, name='max1'))(x)
    x = Dropout(0.5)(x)

    x = TimeDistributed(Conv2D(filters=32, kernel_size=5, strides=(2, 2),
                               padding='same', activation='relu'))(x)
    x = TimeDistributed(MaxPooling2D(pool_size=(2,2), strides=None, name='max1'))(x)
    x = Dropout(0.5)(x)

    x = TimeDistributed(Conv2D(filters=4, kernel_size=5, strides=(2, 2),
                               padding='same', activation='relu'))(x)
    x = TimeDistributed(MaxPooling2D(pool_size=(2,2), strides=None, name='max1'))(x)
    x = Dropout(0.5)(x)

    input_lstm = TimeDistributed(Flatten())(x)

    x_lstm = Bidirectional(GRU(256, return_sequences=True, kernel_initializer='Orthogonal', name='gru1'), merge_mode='concat')(input_lstm)
    x_lstm = Dense(output_size, kernel_initializer='he_normal', name='dense1')(x_lstm)
    print("after dense1")
    y_pred = Activation('softmax', name='softmax')(x_lstm)

    labels = Input(name='the_labels', shape = [max_string_len], dtype='int32')
    input_length = Input(name = 'input_length', shape =[1], dtype = 'int32')
    label_length = Input(name = 'label_length', shape = [1], dtype = 'int32')
    loss = CTC('ctc',[y_pred, labels, input_length, label_length])
    model = Model(inputs=[input_data, labels, label_length, input_length],
                  outputs = loss)
    model.summary()
    # Build model here...

    return model
def pad_labels(labels, max_string_len):
    padding = np.ones((labels.shape[0], max_string_len - labels.shape[1])) * -1
    return np.concatenate((labels, padding), axis = 1)

def train(model, x_train, y_train, label_len_train, input_len_train, batch_size=256, epochs=100, val_train_ratio=0.2):
    max_string_len = 10
    if y_train.shape[1] != max_string_len:
        y_train = pad_labels(y_train, max_string_len)

    adam = Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
    model.compile(loss={'ctc': lambda y_true, y_pred: y_pred}, optimizer=adam)
    history = model.fit(x = {'the_input':x_train, 'the_labels':y_train, 'label_length':label_len_train,
                             'input_length':input_len_train}, y = {'ctc': np.zeros([x_train.shape[0]])},
                        batch_size=batch_size,
                        epochs=epochs,
                        validation_split=val_train_ratio,
                        shuffle=True,
                        verbose=1)

    return history

def read_data():
    oh = OneHotEncoder()
    le = LabelEncoder()

    x = list()
    y = list()
    t = list()
    print("loading images...")
    for i, (img, words) in enumerate(load_data(DATA_PATH, verbose=False, framebyframe=False)):
        if img.shape[0] != 75:
            continue
        x.append(img)
        y.append(words)

        t += words.tolist()
        if i == 3:
            break

    t = le.fit_transform(t)
    oh.fit(t.reshape(-1, 1))

    print("convering to np array...")
    x = np.stack(x, axis=0)

    print("transforming y...")
    for i in range(len(y)):
        y_ = le.transform(y[i])
        y[i] = np.asarray(oh.transform(y_.reshape(-1, 1)).todense())
    y = np.stack(y, axis=0)

    return x, y

def main():
    epochs = 100

    start = time.time()
    print("loading data")
    x, y, label_len, input_len = load_data(DATA_PATH, verbose=True, num_samples=18, ctc_encoding=True)
    end = time.time()

    print("load data took", end-start)
    print("training data shapes:", x.shape, y.shape)
    x_train, x_test, y_train, y_test, label_len_train, label_len_test, \
    input_len_train, input_len_test = train_test_split(x, y, label_len, input_len, test_size=0.2)

    model = build_model(x.shape[1:], 28, max_string_len = 10)

    history = train(model, x_train, y_train, label_len_train, input_len_train, epochs=epochs)

    print("Saving model...")
    model.save('model.h5')

    # TODO: add visualization
#     print("Plotting...")
    #f, (ax1, ax2) = plt.subplots(2, 1)
    #ax1.plot(range(1, epochs+1), history.history['val_acc'], 'tab:blue', label="validation accuracy")
    #ax1.plot(range(1, epochs+1), history.history['acc'], 'tab:red', label="training accuracy")

    #ax2.plot(range(1, epochs+1), history.history['loss'], 'tab:orange', label="loss")
    #ax2.plot(range(1, epochs+1), history.history['val_loss'], 'tab:green', label="validation loss")

    #ax1.legend()
    #ax2.legend()

    #f.savefig('training.png', dpi=300)
    print("Done.")


if __name__ == "__main__":
    main()


loading data
reading: /home/ubuntu/assignments/machine-lip-reading/preprocessing/../data/s1/video/bbifzp.mpg
0: 16--21: bin
1: 21--26: blue
2: 26--28: in
3: 28--32: f
4: 32--40: zero
5: 40--51: please
reading: /home/ubuntu/assignments/machine-lip-reading/preprocessing/../data/s1/video/bbizzn.mpg
6: 16--22: bin
different size, skip
6: 22--27: blue
different size, skip
6: 27--31: in
different size, skip
6: 31--36: z
different size, skip
6: 36--45: zero
different size, skip
6: 45--51: now
different size, skip
reading: /home/ubuntu/assignments/machine-lip-reading/preprocessing/../data/s1/video/bbal8p.mpg
6: 22--27: bin
7: 27--31: blue
8: 31--32: at
9: 32--36: l
10: 36--40: eight
11: 40--50: please
reading: /home/ubuntu/assignments/machine-lip-reading/preprocessing/../data/s1/video/bbbf9a.mpg
12: 22--27: bin
13: 27--31: blue
14: 31--35: by
15: 35--39: f
16: 39--45: nine
17: 45--54: again
0: (11, 50, 100, 3)
Added: (11, 50, 100, 3)
1: (11, 50, 100, 3)
Added: (11, 50, 100, 3)
2: (11, 50, 100,

Epoch 19/100
11/11 [==============================] - 0s 29ms/step - loss: 14.9339 - val_loss: 9.5007
Epoch 20/100
11/11 [==============================] - 0s 29ms/step - loss: 14.9661 - val_loss: 9.4845
Epoch 21/100
11/11 [==============================] - 0s 29ms/step - loss: 14.9531 - val_loss: 9.4676
Epoch 22/100
11/11 [==============================] - 0s 29ms/step - loss: 14.9021 - val_loss: 9.4502
Epoch 23/100
11/11 [==============================] - 0s 28ms/step - loss: 14.8589 - val_loss: 9.4319
Epoch 24/100
11/11 [==============================] - 0s 29ms/step - loss: 14.8814 - val_loss: 9.4122
Epoch 25/100
11/11 [==============================] - 0s 28ms/step - loss: 14.8313 - val_loss: 9.3908
Epoch 26/100
11/11 [==============================] - 0s 29ms/step - loss: 14.8337 - val_loss: 9.3698
Epoch 27/100
11/11 [==============================] - 0s 29ms/step - loss: 14.7561 - val_loss: 9.3487
Epoch 28/100
11/11 [==============================] - 0s 28ms/step - loss: 14.6977

Epoch 100/100
11/11 [==============================] - 0s 29ms/step - loss: 13.7163 - val_loss: 8.5647
Saving model...
Done.


In [1]:
from data import load_data
CURRENT_PATH = '/home/ubuntu/assignments/machine-lip-reading/preprocessing'
DATA_PATH = CURRENT_PATH + '/../data'
x, y, label_len, input_len = load_data(DATA_PATH, verbose=True, num_samples=12, ctc_encoding=True)

new speaker added
('s1', 's1')
reading: /home/ubuntu/assignments/machine-lip-reading/preprocessing/../data/s1/video/bbifzp.mpg
0: 16--21: bin
<type 'str'>
1: 21--26: blue
<type 'str'>
2: 26--28: in
<type 'str'>
3: 28--32: f
<type 'str'>
4: 32--40: zero
<type 'str'>
5: 40--51: please
<type 'str'>
reading: /home/ubuntu/assignments/machine-lip-reading/preprocessing/../data/s1/video/bbizzn.mpg
6: 16--22: bin
different size, skip
6: 22--27: blue
different size, skip
6: 27--31: in
different size, skip
6: 31--36: z
different size, skip
6: 36--45: zero
different size, skip
6: 45--51: now
different size, skip
reading: /home/ubuntu/assignments/machine-lip-reading/preprocessing/../data/s1/video/bbal8p.mpg
6: 22--27: bin
<type 'str'>
7: 27--31: blue
<type 'str'>
8: 31--32: at
<type 'str'>
9: 32--36: l
<type 'str'>
10: 36--40: eight
<type 'str'>
11: 40--50: please
<type 'str'>
0: (11, 50, 100, 3)
Added: (11, 50, 100, 3)
1: (11, 50, 100, 3)
Added: (11, 50, 100, 3)
2: (11, 50, 100, 3)
Added: (11, 50,

IOError: [Errno 2] No such file or directory: '/home/ubuntu/assignments/machine-lip-reading/preprocessing/../data/numpy/s1x.npy'

In [4]:
import numpy as np
np.load('test_savex.npy')

array([[[[[225., 185., 149.],
          [229., 179., 148.],
          [228., 176., 146.],
          ...,
          [109., 132., 149.],
          [ 77., 141., 161.],
          [ 64., 144., 169.]],

         [[226., 185., 149.],
          [228., 179., 146.],
          [226., 175., 143.],
          ...,
          [ 84., 134., 160.],
          [ 62., 147., 172.],
          [ 54., 149., 177.]],

         [[227., 184., 149.],
          [228., 180., 144.],
          [225., 176., 140.],
          ...,
          [ 61., 145., 181.],
          [ 43., 154., 185.],
          [ 39., 154., 186.]],

         ...,

         [[ 69.,  57.,  45.],
          [ 98.,  74.,  53.],
          [174., 148., 125.],
          ...,
          [ 48., 144., 156.],
          [ 35., 142., 170.],
          [ 34., 141., 170.]],

         [[ 67.,  55.,  44.],
          [ 93.,  69.,  50.],
          [164., 138., 117.],
          ...,
          [ 52., 144., 154.],
          [ 37., 143., 168.],
          [ 36., 142., 168.]],



In [5]:
np.load('test_savey.npy')

array([[ 1.,  8., 13., -1., -1., -1.],
       [ 1., 11., 20.,  4., -1., -1.],
       [ 8., 13., -1., -1., -1., -1.],
       [ 5., -1., -1., -1., -1., -1.],
       [25.,  4., 17., 14., -1., -1.],
       [15., 11.,  4.,  0., 18.,  4.],
       [ 1.,  8., 13., -1., -1., -1.],
       [ 1., 11., 20.,  4., -1., -1.],
       [ 0., 19., -1., -1., -1., -1.],
       [11., -1., -1., -1., -1., -1.],
       [ 4.,  8.,  6.,  7., 19., -1.],
       [15., 11.,  4.,  0., 18.,  4.],
       [ 1.,  8., 13., -1., -1., -1.],
       [ 1., 11., 20.,  4., -1., -1.],
       [ 1., 24., -1., -1., -1., -1.],
       [ 5., -1., -1., -1., -1., -1.],
       [13.,  8., 13.,  4., -1., -1.],
       [ 0.,  6.,  0.,  8., 13., -1.]])

In [3]:
checks = ["s1", "s", "abxs", "s2", "s2345", "abs234"]
import re
pattern = re.compile("s[0-9]")
for i in checks:
    match = pattern.match(i)
    print(match)
    

None
None
None


In [2]:
speakers = []
speakers.append("s1")
speakers.append("s2")
speakers.append("s3")
speakers

['s1', 's2', 's3']

In [17]:
import numpy as np
word_dict = {}
word_dict["foo"] = []
np1 = np.array([[1,2],[3,4]])
word_dict["foo"].append(np1)
np2 = np.array([[4,5],[6,7]])
word_dict["foo"].append(np2)  
# word_dict["foo"].append("bang")
m = {'x':np.array([1,2,3])}
# type(m)
type(word_dict)
word_dict

{'foo': [array([[1, 2],
         [3, 4]]), array([[4, 5],
         [6, 7]])]}